In [1]:
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig, CacheMode
import asyncio
import sys
from threading import Thread
from queue import Queue

# Force Windows to use SelectorEventLoop instead of ProactorEventLoop
if sys.platform == 'win32':
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

urls_to_crawl = [
    "https://docs.llamaindex.ai/en/stable/understanding/",
]

config = CrawlerRunConfig(
    cache_mode=CacheMode.BYPASS,
    page_timeout=80000,
    word_count_threshold=50,
)

async def crawl_website():
    data_res = {"data": []}

    async with AsyncWebCrawler(headless=True, verbose=True) as crawler:
        results = await crawler.arun_many(urls_to_crawl, config=config)

        for result in results:
            if result.success:
                title = result.metadata.get("title", "")
                if not title and result.markdown:
                    lines = result.markdown.raw_markdown.split("\n")
                    for line in lines:
                        if line.startswith("#"):
                            title = line.strip("#").strip()
                            break

                data_res["data"].append(
                    {
                        "text": result.markdown.raw_markdown if result.markdown else "",
                        "meta": {"url": result.url, "meta": {"title": title}},
                    }
                )

    return data_res

def run_in_thread(queue):
    """Run the async function in a new thread with its own event loop"""
    if sys.platform == 'win32':
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        result = loop.run_until_complete(crawl_website())
        queue.put(("success", result))
    except Exception as e:
        queue.put(("error", str(e)))
    finally:
        loop.close()

# Execute in a separate thread
result_queue = Queue()
thread = Thread(target=run_in_thread, args=(result_queue,))
thread.start()
thread.join()

# Get the result
status, data_res = result_queue.get()
if status == "success":
    print(data_res)
else:
    print(f"Error: {data_res}")

Task exception was never retrieved
future: <Task finished name='Task-23' coro=<Connection.run() done, defined at c:\DROPBOX\Документы\Карзановы\Андрей\__CHALLENGES\AI_Full_Stack_Developer\.venv\Lib\site-packages\playwright\_impl\_connection.py:303> exception=NotImplementedError()>
Traceback (most recent call last):
  File "c:\DROPBOX\Документы\Карзановы\Андрей\__CHALLENGES\AI_Full_Stack_Developer\.venv\Lib\site-packages\playwright\_impl\_connection.py", line 310, in run
    await self._transport.connect()
  File "c:\DROPBOX\Документы\Карзановы\Андрей\__CHALLENGES\AI_Full_Stack_Developer\.venv\Lib\site-packages\playwright\_impl\_transport.py", line 133, in connect
    raise exc
  File "c:\DROPBOX\Документы\Карзановы\Андрей\__CHALLENGES\AI_Full_Stack_Developer\.venv\Lib\site-packages\playwright\_impl\_transport.py", line 120, in connect
    self._proc = await asyncio.create_subprocess_exec(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<9 lines>...
    )
    ^
  File "C:\

Error: 
